In [6]:
import pandas as pd

In [7]:
import time

In [8]:
pd.set_option('display.max_columns', 100)

empirically, loading data from "signed_predictors_dl_wide.csv" takes 2 mins

loading data from "sp500_op_ret.csv" takes 10 secs

I am using macbook pro with intel i7, 16GB memory

In [9]:
start = time.time()
signed_predictors_dl_wide = pd.read_csv("data/signed_predictors_dl_wide.csv") 
print(time.time() - start)

98.4389419555664


In [11]:
start = time.time()
sp500_op_ret = pd.read_csv("data/sp500_op_ret.csv")
print(time.time() - start)

10.234528064727783


In [12]:
mapping_table = pd.read_csv("data/mapping_table.csv")

In [13]:
sp500_const_list = pd.read_csv("data/sp500_const_list.csv")

In [14]:
sp500_op_ret = sp500_op_ret.merge(mapping_table, on="secid", how="left")

In [15]:
# filter sparse entries

In [16]:
na_freq_dict = dict(
    signed_predictors_dl_wide.isna().sum() / signed_predictors_dl_wide.shape[0])

In [17]:
[key for key, val in na_freq_dict.items() if val < 0.2]

['permno',
 'yyyymm',
 'Beta',
 'BidAskSpread',
 'Coskewness',
 'DolVol',
 'High52',
 'IdioRisk',
 'IdioVol3F',
 'IdioVolAHT',
 'MaxRet',
 'PriceDelayRsq',
 'PriceDelaySlope',
 'PriceDelayTstat',
 'ReturnSkew',
 'ReturnSkew3F',
 'VolMkt',
 'zerotrade',
 'zerotradeAlt1']

In [23]:
import datetime

In [18]:
signed_predictors_dl_wide = signed_predictors_dl_wide[
    [key for key, val in na_freq_dict.items() if val < 0.2]
]

In [32]:
signed_predictors_dl_wide["yyyymm"].iloc[:3].apply(
    lambda x: datetime.datetime.strptime(str(x), '%Y%m'))

0   1986-01-01
1   1986-02-01
2   1986-03-01
Name: yyyymm, dtype: datetime64[ns]

In [38]:
signed_predictors_dl_wide["date"] = signed_predictors_dl_wide["yyyymm"].apply(
    lambda x: datetime.datetime.strptime(str(x), '%Y%m'))

In [39]:
sp500_op_ret["date2"] = sp500_op_ret.date.apply(
    lambda x: datetime.datetime.strptime(str(x)[:7], '%Y-%m'))

In [48]:
df_merged = sp500_op_ret.merge(
    signed_predictors_dl_wide, 
    left_on=["permno", "date2"], right_on=["permno", "date"],
    how="left"
)

In [49]:
df_merged

,secid,date_x,exdate,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,optionid,cfadj,days_no_trading,days_to_exp,forwardprice,spotprice,adj_spot,ir_rate,mid_price,option_ret,sdate,edate,permno,date2,yyyymm,Beta,BidAskSpread,Coskewness,DolVol,High52,IdioRisk,IdioVol3F,IdioVolAHT,MaxRet,PriceDelayRsq,PriceDelaySlope,PriceDelayTstat,ReturnSkew,ReturnSkew3F,VolMkt,zerotrade,zerotradeAlt1,date_y
0,5015,1996-01-31 00:00:00,1996-07-20 00:00:00,C,20.0,4.625,5.000,15,25,0.380062,0.793831,0.044693,4.456725,-2.603545,10980421,1,0,171,24.142672,24.000,24.0000,0.052253,4.8125,-0.025367,1996-01-01 00:00:00,1998-10-01 00:00:00,56266,1996-01-01,199601.0,0.744864,0.003750,-0.276376,-5.460070,0.864865,-0.032237,-0.027134,-0.018821,-0.045714,0.041281,0.389311,1.212702,1.425051,1.314407,-0.083688,2.212753e-08,2.905242e-08,1996-01-01
1,5048,1996-01-31 00:00:00,1996-08-17 00:00:00,P,40.0,3.000,3.375,5,37,0.245716,-0.507195,0.058756,11.186040,-1.508168,11613022,1,0,199,39.344930,39.000,39.0000,0.051780,3.1875,0.002347,1996-01-01 00:00:00,1999-04-01 00:00:00,27051,1996-01-01,199601.0,0.267191,0.006422,-0.140841,-6.200769,0.866667,-0.016901,-0.015479,-0.014096,-0.027027,0.108268,-0.962344,0.876436,0.584634,0.771925,-0.057416,3.198582e-08,3.837764e-08,1996-01-01
2,5049,1996-01-31 00:00:00,1996-05-18 00:00:00,C,65.0,5.875,6.375,4,420,0.167298,0.833206,0.039200,9.020193,-5.515290,11618802,1,0,108,70.168661,70.375,35.1875,0.053274,6.1250,-0.009721,1996-01-01 00:00:00,1998-12-31 00:00:00,19553,1996-01-01,199601.0,0.001779,0.002013,0.119932,-7.043676,0.975737,-0.010260,-0.007803,-0.008903,-0.010909,0.113548,-0.210562,1.205771,1.263040,0.543148,-0.031556,6.169017e-08,5.649152e-08,1996-01-01
3,5049,1996-01-31 00:00:00,1996-08-17 00:00:00,C,70.0,3.250,3.625,13,3806,0.149214,0.559395,0.050652,19.823520,-4.653782,11770703,1,0,199,70.393348,70.375,35.1875,0.051780,3.4375,0.001186,1996-01-01 00:00:00,1998-12-31 00:00:00,19553,1996-01-01,199601.0,0.001779,0.002013,0.119932,-7.043676,0.975737,-0.010260,-0.007803,-0.008903,-0.010909,0.113548,-0.210562,1.205771,1.263040,0.543148,-0.031556,6.169017e-08,5.649152e-08,1996-01-01
4,5061,1996-01-31 00:00:00,1996-06-22 00:00:00,C,30.0,3.375,3.750,5,254,0.324523,0.657941,0.056807,7.146084,-3.929316,10914516,1,0,143,32.001209,31.750,31.7500,0.052726,3.5625,-0.006649,1996-01-01 00:00:00,1999-11-17 00:00:00,10364,1996-01-01,199601.0,0.487953,0.006490,0.300095,-4.950348,0.869863,-0.018204,-0.015692,-0.017747,-0.058333,0.100165,0.294793,0.762450,-0.714502,-0.844009,-0.117665,1.901262e-08,1.398601e-08,1996-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3982031,214905,2021-11-30 00:00:00,2022-07-15 00:00:00,C,13.0,1.150,1.500,134,665,0.437066,0.487561,0.095777,3.813131,-1.399446,143779767,1,0,227,12.112223,12.310,12.3100,0.002822,1.3250,-0.068384,2020-11-17 00:00:00,2020-12-31 00:00:00,20057,2021-11-01,202111.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-01
3982032,214905,2021-11-30 00:00:00,2022-07-15 00:00:00,C,15.0,0.100,0.750,5,14,0.335686,0.249430,0.098387,3.066143,-0.847491,143779769,1,0,227,12.112223,12.310,12.3100,0.002822,0.4250,0.055073,2020-11-17 00:00:00,2020-12-31 00:00:00,20057,2021-11-01,202111.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-01
3982033,214905,2021-11-30 00:00:00,2022-07-15 00:00:00,P,11.0,0.200,1.100,3,111,0.308935,-0.297605,0.114422,3.328838,-0.815255,143779782,1,0,227,12.112223,12.310,12.3100,0.002822,0.6500,0.020419,2020-11-17 00:00:00,2020-12-31 00:00:00,20057,2021-11-01,202111.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-01
3982034,214905,2021-11-30 00:00:00,2022-07-15 00:00:00,P,12.0,1.100,1.600,53,35,0.372075,-0.422035,0.106850,3.744469,-1.102309,143779783,1,0,227,12.112223,12.310,12.3100,0.002822,1.35

In [42]:
len(set(sp500_op_ret["date2"]) - set(signed_predictors_dl_wide["date"]))

0

In [41]:
len(set(signed_predictors_dl_wide["date"]))

1159

In [45]:
df1 = pd.DataFrame({
    "f1": [1, 1, 1],
    "time": [1,2,3]
})

df2 = pd.DataFrame({
    "f1": [1, 1, 1],
    "time": [1, 2, 3],
    "feature": [0,9,8]
})

In [47]:
df1.merge(df2, left_on=["f1", "time"], right_on = ["f1", "time"], how="left")

,f1,time,feature
0,1,1,0
1,1,2,9
2,1,3,8


In [ ]:
df1.merge(df2, on="f1", how="left")

,f1,time_x,time_y,feature
0,1,1,1,0
1,1,1,2,9
2,1,1,3,8
3,1,2,1,0
4,1,2,2,9
5,1,2,3,8
6,1,3,1,0
7,1,3,2,9
8,1,3,3,8


In [15]:
sp500_op_permno = set(sp500_op_ret["permno"])

In [ ]:
sp500_op_ret.merge(
    signed_predictors_dl_wide[signed_predictors_dl_wide["permno"].isin(sp500_op_permno)],
    on="permno", how="left"
)

In [14]:
len(set(signed_predictors_dl_wide["permno"]))

34424

In [12]:
# merge on permno 

In [ ]:
sp500_op_ret.merge(signed_predictors_dl_wide, on="permno", how="inner")

In [12]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [43]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [54]:
from sklearn import linear_model